# Panoramsko spajanje slika

Kako iskombinovati mnostvo slika i kreirati jednu vecu sliku - panoramu.

Ovaj problem sastoji se od nekoliko koraka:
1. Izdvanjanje skupa zajednickih tacaka. odnosno prepozanvanje geometrijske veze izmedju slika (na primer SIFT detektorom)
<br><img src="images/feature_extraction.png" alt="Feature extraction" width="700"/>

2. Izracunavanje homografije. Homografija je transformacija kojom se jedna slika prebacuje u svet druge slike. Prakticno poravnanje slika.
Nakon ovog koraka dobiju se slike koje su "nadovezane" jedna na drugu.
<br><img src="images/stiched_image.png" alt="Feature extraction" width="700"/>

3. Stapanje slika, jer slike skoro nikada nece biti slikane sa istom ekspozicijom
<br><br><img src="images/blending.png" alt="Feature extraction" width="500"/>

## Racunanje homografije

Homografija je matrica transformacija koja prevodi jednu ravan u drugu ravan kroz tacku projekcije (tacka kamere).
<br><br><img src="images/homography_composition.png" alt="Feature extraction" width="500"/>

Pretpostavka koju moramo uvesti ovde je da:
1. Tacke scene leze na istoj ravni
2. Tacke su slikane iz iste tacke (tacke kamere)
3. **Tacke su veoma udaljene** (sto je najcesci slucaj kada su u pitanju panorame, jer retko ce slike biti uslikane bez imalo pomeranja kamere)

### Postavka problema

Ako imamo tacku Ps koja predstavlja tacku izvora sa prve slike i Pd koje je tacka destinacije na drugoj slici, tada imamo projektivno preslikavanje:

\begin{bmatrix}x_d \\ y_d \\ 1 \end{bmatrix} = \begin{bmatrix}\tilde{x_d} \\ \tilde{y_d} \\ \tilde{z_d} \end{bmatrix} = \begin{bmatrix}h_{11} && h_{12} && h_{13} \\ h_{21} && h_{22} && h_{23} \\ h_{31} && h_{32} && h_{33} \end{bmatrix} \begin{bmatrix}x_s \\ y_s \\ 1 \end{bmatrix}

Minimum parova tacaka koji nam je potreban za izracunavanje homografije je 4 (jer zbog osobina homogenih koordinata mozemo da eliminisemo jednu pa imamo 8 nepoznathih), ali uvek zelimo da upotrebimo sto vise kako bismo dobili sto precizniju homografiju.

Za par tacaka sa indeksom i:

$$
\displaystyle x_d^{(i)} = \frac{\tilde{x_d}^{(i)}}{\tilde{z_d}^{(i)}} = \frac{h_{11}x_s^{(i)} + h_{12}y_s^{(i)} + h_{13}}{h_{31}x_s^{(i)} + h_{32}y_s^{(i)} + h_{33}}
$$

$$
\displaystyle y_d^{(i)} = \frac{\tilde{y_d}^{(i)}}{\tilde{z_d}^{(i)}} = \frac{h_{21}x_s^{(i)} + h_{22}y_s^{(i)} + h_{23}}{h_{31}x_s^{(i)} + h_{32}y_s^{(i)} + h_{33}}
$$

Zapisano u malo drugacijem formatu:

$$
\displaystyle x_d^{(i)} (h_{31}x_s^{(i)} + h_{32}y_s^{(i)} + h_{33}) = h_{11}x_s^{(i)} + h_{12}y_s^{(i)} + h_{13}
$$

$$
\displaystyle y_d^{(i)} (h_{31}x_s^{(i)} + h_{32}y_s^{(i)} + h_{33} )= h_{21}x_s^{(i)} + h_{22}y_s^{(i)} + h_{23}
$$

I zapisano u matricnom obliku:
\begin{bmatrix}x_s^{(i)} && y_s^{(i)} && 1 && 0 && 0 && 0 && -x_d{(i)}x_s{(i)} && -x_d{(i)}y_s{(i)} && -x_d{(i)} \\ 0 && 0 && 0 && x_s^{(i)} && y_s^{(i)} && 1 && -y_d{(i)}x_s{(i)} && -y_d{(i)}y_s{(i)} && -y_d{(i)} \end{bmatrix} \begin{bmatrix} h_{11} \\ h_{12} \\ h_{13} \\h_{21} \\ h_{22} \\ h_{23} \\ h_{31} \\ h_{32} \\ h_{33}\end{bmatrix} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}

Ako to ukombinujemo sa svih datih n tacaka:
\begin{bmatrix}x_s^{(1)} && y_s^{(1)} && 1 && 0 && 0 && 0 && -x_d{(1)}x_s{(1)} && -x_d{(1)}y_s{(1)} && -x_d{(1)} \\ 0 && 0 && 0 && x_s^{(1)} && y_s^{(1)} && 1 && -y_d{(1)}x_s{(1)} && -y_d{(1)}y_s{(1)} && -y_d{(1)} \\ \vdots \\ x_s^{(i)} && y_s^{(i)} && 1 && 0 && 0 && 0 && -x_d{(i)}x_s{(i)} && -x_d{(i)}y_s{(i)} && -x_d{(i)} \\ 0 && 0 && 0 && x_s^{(i)} && y_s^{(i)} && 1 && -y_d{(i)}x_s{(i)} && -y_d{(i)}y_s{(i)} && -y_d{(i)} \\ \vdots \\ x_s^{(n)} && y_s^{(n)} && 1 && 0 && 0 && 0 && -x_d{(n)}x_s{(n)} && -x_d{(n)}y_s{(n)} && -x_d{(n)} \\ 0 && 0 && 0 && x_s^{(n)} && y_s^{(n)} && 1 && -y_d{(n)}x_s{(n)} && -y_d{(n)}y_s{(n)} && -y_d{(n)}\end{bmatrix} \begin{bmatrix} h_{11} \\ h_{12} \\ h_{13} \\h_{21} \\ h_{22} \\ h_{23} \\ h_{31} \\ h_{32} \\ h_{33}\end{bmatrix} = \begin{bmatrix} 0 \\ 0 \\ \vdots \\ 0 \\ 0 \\ \vdots \\ 0 \\ 0\end{bmatrix}

Pa sustinski treba resiti $A h = 0$, sa dodatim uslovom da je $||h||^2 = 1$



### Izracunavanje $A h = 0$, kada je $||h||^2 = 1$

Resava se metodom najmanjih kvadrata sa organicenjem.
Definicija problema:

$$
\displaystyle \min\limits_{h} ||Ah||^2 \text{ tako da je } \displaystyle ||h||^2 = 1
$$

Ono sto znamo je da:

$$
\displaystyle ||Ah||^2 = (Ah)^T(Ah) = h^TA^TAh \text{ i } ||h||^2 = h^Th = 1
$$

Tako da ono sto trazimo je zapravo:

$$
\displaystyle \min\limits_{h} (h^TA^TAh) \text{ tako da je } \displaystyle h^Th = 1
$$

Za ovo nam je potreban metod Lagranzevih multiplikatora. Osnovna ideja je da se problem sa organicenjem svede na neogranicen problem kako bi se mogli primeniti testovi pomocu izvoda da bi se nasao minimum. Veza izmedju gradijenta funkcije i organicenja u opstem obliku izgleda ovako:

$$
\displaystyle {\mathcal {L}}(x,\lambda )\equiv f(x)+\langle \lambda ,g(x)\rangle
$$

i vrednost **$\lambda$** se zove upravo Lagranzev multiplikator.

U nasem slucaju, funkcija greske izgleda ovako:

$$
\displaystyle {\mathcal {L}}(h,\lambda )\equiv h^TA^TAh + \lambda\langle h^Th - 1 \rangle
$$

Kada izracunamo prvi izvod ove funkcije po h i izjednacimo sa 0 kako bismo dobili stacionarne tacke funkcije:

$$
2A^TAh - 2\lambda h = 0
$$

odnosno

$$
2A^TAh = 2\lambda h
$$

sto je zapravo problem sopstvenih vrednosti i sopstvenih vektora matrice A.

**Konacno, sopstveni vektor h sa najmanjom sopstvenom vrednoscu $\lambda$ matrice $A^TA$ minimizuje funkciju greske $L(h)$.**

Kada se vektor h zapise kao 3x3 matrica dobija se trazena homografija.

### Sta raditi sa anomalijama medju parovima tacaka?

Vrlo cesto ce nam algoritmi za prepoznavanje pored odgovarajucih tacaka dati i neke pogresne rezultate koji nam mogu pokvariti rezultat.
Sa njima se takodje treba pozabaviti i za to se koristi tehnika koja se zove **RANSAC** (Random sample consensus) algoritam.
RANSAC algoritam postoji jos od ranih 80-tih godina proslog veka i ima razne upotrebe. Veoma je mocan - moze se desiti da postoji veliki broj anomalija medju tackama i on ce se odlicno izboriti, sve dok broj anomalija ne prelazi 50% ukupnog broja tacaka.

Koraci u RANSAC algoritmu:
1. Nasumicno odabrati S tacaka
2. Ubaciti nasumicno odabrane tacke u model
3. Definisati prihvatljivu gresku $\epsilon$ i prebrojati koji broj tacaka - M ce primenom modela imati gresku manju od $\epsilon$
4. Ponavljati korake 1-3 N puta
5. Odabrati onaj model koji ima najveci broj M

U primeru homografije:
1. Odaberemo 4 tacke jer znamo da je to potreban minimum za formiranje modela
2. Formiramo matricu homografije sa dobijenim tackama h
3. Gresku $\epsilon$ definisemo kao rastojanje od stvarne tacke i onu koju nam da model. Primenimo matricu h na svaku tacku i izracunamo rastojanje izmedju rezultata i onoga sto smo zapravo dobili kao par za tu tacku. Izbrojimo koliko imamo tacaka koje su ovim kriterijumom oznacene kao ispravne i tako dobijemo M.
4. Ponavljamo korake 1-3 N puta
5. Odaberemo matricu koja je dala najveci broj za M

## Geometrijska transformacija

Medju slikama na ulazu biracemo jednu sliku koja ce nam biti referenca i na sve ostale cemo primeniti izracunatu homografiju kako bismo ostale slike preveli u koordinatni sistem referentne slike.
Medjutim, i tu postoji nekoliko tehnickih problema.

Ako imamo transformaciju T i sliku $f(x, y)$ i hocemo da izracunamo transformisanu sliku $g(x, y)$:

$$
g(x,y)=f(T(x,y))
$$

Sto deluje jednostavno. Medjutim sa tehnicke strane postoje dva problema:
1. Sta se desi ako je rezultujuci piksel izmedju dva piksela? Mozemo uzeti boju najblizeg suseda ili raditi interpolaciju suseda da dobijemo odgovarajucu boju.
2. Sta se desava ako nisu svi pikseli u mapiranju pokriveni? Mogu postojati rupe u $g(x,y)$. Ovaj problem je nezgodniji za resavanje.

Da resimo problem rupa ne radimo direktnu nego suprotnu transformaciju $T^{-1}$.

To radimo tako sto prvo iz prve slike uzmemo coskove i mapiramo ih pomocu T.
Time dobijamo okvir slike $g(x, y)$ i koordinate piksela slike g. Na njih primenjujemo inverznu transformaciju $T^{-1}$.


## Stapanje slika

Kada se nakon svega slike poravnaju ono sto ce se desiti jeste da ce se videti linije, odnosno ivice na mestima gde se slike spajaju koje dolaze od razlika u svetlosti, ekspoziciji i slicno.
Potreban nam je neki nacin da stopimo slike tako da se ova razlika ne vidi.

Resenje ovog problema lezi u racunanju tezinskih funkcija koje govore o tome sa kojom tezinom se uzimaju pikseli iz koje slike.
Ako imamo tri slike $I_1, I_2, I_3$ za svaku treba da izracunamo tezinsku funkciju $\omega_1, \omega_2, \omega_3$.
Funkcija koju cemo koristiti: Tezina koju dodeljujemo svakom pikselu je funkcija rastojanja svakog piksela od najblize ivice slike.
Ideja je da pikseli koji su blize ivicama dobiju manje tezine.